# Parte inicial del ETL creado para coderhouse

## Importamos las librerias necesarias

In [1]:
import requests
import json
import pandas as pd

In [2]:
API_KEY = 'f29d2a34d2f0fcf9db7c748ca09c7a1a'
MOVIE_ID = '447404'
# 10859
# https://api.themoviedb.org/3/movie/{MOVIE_ID}/credits?language=es-MX&api_key={API_KEY}
# https://api.themoviedb.org/3/movie/{MOVIE_ID}?language=es-MX&api_key={API_KEY}

In [3]:
credits = requests.get(f'https://api.themoviedb.org/3/movie/{MOVIE_ID}/credits?language=es-MX&api_key={API_KEY}')
movie = requests.get(f'https://api.themoviedb.org/3/movie/{MOVIE_ID}?language=es-MX&api_key={API_KEY}')
if credits.status_code and movie.status_code == 200:
  print('--ok--')

--ok--


In [4]:
c = json.loads(credits.text)
m = json.loads(movie.text)
cred = pd.json_normalize(c)
movi = pd.json_normalize(m)
df_crew = pd.DataFrame(cred['crew'][0])
df_cast = pd.DataFrame(cred['cast'][0])
df_movi = pd.DataFrame(movi)

In [6]:
df_crew

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job
0,False,1,474,Production,Jina Jay,Jina Jay,5.173,None,61c48bd021c4ca0043e56597,Production,Casting
1,False,2,943,Camera,John Mathieson,John Mathieson,7.124,/AkWm67UMEWHhF3BDfH32HkRl5oM.jpg,5aad339b92514116b4012918,Camera,Director of Photography
2,False,1,7232,Production,Sarah Halley Finn,Sarah Halley Finn,13.156,/jxECVz09YGZAg1AmdzfEkPm4xeY.jpg,61c48bd937b3a9008fbf3eea,Production,Casting
3,False,0,10570,Production,Joseph M. Caracciolo Jr.,Joseph M. Caracciolo Jr.,3.664,None,61c48a72b6c264006103657c,Production,Executive Producer
4,False,0,10570,Production,Joseph M. Caracciolo Jr.,Joseph M. Caracciolo Jr.,3.664,None,61c48c4237b3a900c347e840,Production,Unit Production Manager
5,False,1,8532,Crew,Lisa Dennis,Lisa Dennis,0.753,None,61c48da537b3a9004da79e1c,Production,Co-Producer
6,False,2,9817,Art,Nigel Phelps,Nigel Phelps,3.606,None,58cbb18dc3a368285d00efa2,Art,Production Design
7,False,2,17315,Production,Cale Boyter,Cale Boyter,2.962,None,5d05a77792514110b7b85937,Production,Producer
8,False,2,65630,Directing,Rob Letterman,Rob Letterman,4.017,/bgsnUXYPWsRDuo3B0f9wthXKxll.jpg,5c9c836f92514124c830b06f,Writing,Screenplay
9,False,2,65630,Directing,Rob Letterman,Rob Letterman,4.017,/bgsnUXYPWsRDuo3B0f9wthXKxll.jpg,58cbb017c3a368287d01076b,Directing,Director


In [7]:
df_cast

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order
0,False,2,10859,Acting,Ryan Reynolds,Ryan Reynolds,64.800,/4SYTH5FdB0dAORV98Nwg3llgVnY.jpg,120,Detective Pikachu (voice),6058f4d2d5ffcb00562ed261,0
1,False,2,1029934,Acting,Justice Smith,Justice Smith,15.815,/3UoUT44XYWoWNsZaqHzN3w48gZL.jpg,17,Tim Goodman,5a1fafb1c3a3680b8d088f16,1
2,False,1,221192,Acting,Kathryn Newton,Kathryn Newton,46.057,/hEz2R1RTgaNznJ3fa9n3KabIXYl.jpg,16,Lucy Stevens,5a1faf95925141033608a00b,2
3,False,1,1325961,Acting,Suki Waterhouse,Suki Waterhouse,9.724,/bpGRoG2DRsySQBICMzCuvfFDg0e.jpg,20,Ms. Norman,5a7aaa7ec3a3687b6a004263,3
4,False,2,1134885,Acting,Omar Chaparro,Omar Chaparro,4.582,/eucXVp57mAsea5t3oJFD3KPpqKq.jpg,32,Sebastian,5c762880c3a3686c3dd1d47b,4
5,False,2,62879,Acting,Chris Geere,Chris Geere,7.033,/y5Ti23z47KY0XG7lzSLxffB0QKn.jpg,22,Roger Clifford,5a7aaa950e0a26020c003ea3,5
6,False,1,1089873,Acting,Rita Ora,Rita Ora,12.458,/o1DMrW8vsTYnCcvVTZy6hEvlk4f.jpg,23,Dr. Ann Laurent,5aa81ffe0e0a263dca0242c8,6
7,False,2,1109702,Acting,Karan Soni,Karan Soni,20.368,/hkyWd9bcuOqUAUepOJLE47izTWR.jpg,30,Jack,5c7628560e0a260b4c1ca9ba,7
8,False,1,174708,Acting,Josette Simon,Josette Simon,7.939,/uLvLGR6a60To8AAlj0C5124d7Xu.jpg,94,Grams,5cc76dcb0e0a263743efb31b,8
9,False,2,3899,Acting,Ken Watanabe,Ken Watanabe,12.232,/w2t30L5Cmr34myAaUobLoSgsLfS.jpg,21,Lieutenant Hide Yoshida,5a7aaa8b9251411c5200379f,9


In [8]:
df_movi

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/nDP33LmQwNsnPv29GQazz59HjJI.jpg,None,150000000,"[{'id': 28, 'name': 'Acción'}, {'id': 12, 'nam...",https://www.warnerbroslatino.com/es-mx/pelicul...,447404,tt5884052,en,Pokémon Detective Pikachu,...,2019-05-03,449762638,105,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,¡Asociarse!,Pokémon Detective Pikachu,False,6.953,6303


In [9]:
distinct_crew = df_crew['original_name'].nunique()
distinct_cast = df_cast['original_name'].nunique()
total = distinct_crew + distinct_cast
titulo = df_movi.at[0,'title']

In [10]:
print(f'En la pelicula {titulo} trabajaron {total} personas, hay {distinct_cast} actores y {distinct_crew} en el staff')

En la pelicula Pokémon Detective Pikachu trabajaron 74 personas, hay 22 actores y 52 en el staff


In [11]:
director_rows = df_crew[df_crew['job'].isin(['Director', 'Executive Producer', 'Producer'])]

# Agrupar por 'job' y tomar la primera fila de cada grupo
first_rows = director_rows.groupby('job').first()
first_rows

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department
job,,,,,,,,,,
Director,False,2,65630,Directing,Rob Letterman,Rob Letterman,4.017,/bgsnUXYPWsRDuo3B0f9wthXKxll.jpg,58cbb017c3a368287d01076b,Directing
Executive Producer,False,0,10570,Production,Joseph M. Caracciolo Jr.,Joseph M. Caracciolo Jr.,3.664,None,61c48a72b6c264006103657c,Production
Producer,False,2,17315,Production,Cale Boyter,Cale Boyter,2.962,None,5d05a77792514110b7b85937,Production
